In [1]:
import ray
from ray import tune
import time

In [2]:
f=open('../head_node_info','r+')
buf=f.readlines()
buf=buf[0].strip().split()
address=buf[0]
node=address.split(':')[0]
redis_pass=buf[1]
f.close()

In [3]:
ray.init(address=address,_redis_password=redis_pass)

2023-02-13 00:16:00,340	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 10.109.75.157:45021...
2023-02-13 00:16:00,376	INFO worker.py:1529 -- Connected to Ray cluster. View the dashboard at 10.109.75.157:33694 


Python version:,3.9.16
Ray version:,2.2.0
Dashboard:,http://10.109.75.157:33694


In [4]:
def objective(x, a, b):
    return a * (x ** 0.5) + b

In [5]:
class trainable (tune.Trainable):
    def setup(self,config):
        self.x = 0
        self.a = config['a']
        self.b = config['b']
      
  # step will be called iteratively
    def step(self):
        score = objective (self.x, self.a, self.b)
        self.x += 1
        return {"score": score}

In [ ]:
analysis = tune.run(
    trainable,
    num_samples=20,
    stop={"training_iteration": 20},
    resources_per_trial={"cpu":1},
    config={"a": tune.uniform(0, 20),
            "b": tune.uniform(0,20)},
    max_concurrent_trials=8 )

In [ ]:
print("Best config is:", analysis.get_best_config(metric="score", mode="max"))

In [ ]:
!ps -eF | grep ray